In [1]:
#使用ResNet18初始化，使用自定义模型训练（多次多头注意力迭代）
#更改Wx输出后的大小
#只加入注意力-并

In [2]:
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast
import warnings
warnings.filterwarnings("ignore")

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\qyxx-mv3.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\qyxx-mv3.pkl"  #修改此处文件名

In [5]:
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
from torchvision import models
mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)

mobilenet_v3_large

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

In [8]:
class hswish(nn.Module):
    def forward(self, x):
        out = x * F.relu6(x + 3, inplace=True) / 6
        return out
class SKNet(nn.Module):
    def __init__(self, num_class=7):
        super(SKNet, self).__init__()
        self.features = nn.Sequential(*list(mobilenet_v3_large.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(960, 1280),
            hswish(),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, num_class)
        )
    def forward(self, x):
        
        x = self.features(x)
        out = self.avgpool(x)
        out = torch.flatten(out,1)
        out = self.fc(out)
        
        return out

In [9]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [10]:
#to(device)将模型加入GPU中加速计算
model = SKNet().to(device)
#设置优化器
params = list(model.parameters()) 
optimizer = optim.AdamW(params, lr=lr)

# optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 16, 112, 112]             432
       BatchNorm2d-2        [128, 16, 112, 112]              32
         Hardswish-3        [128, 16, 112, 112]               0
            Conv2d-4        [128, 16, 112, 112]             144
       BatchNorm2d-5        [128, 16, 112, 112]              32
              ReLU-6        [128, 16, 112, 112]               0
            Conv2d-7        [128, 16, 112, 112]             256
       BatchNorm2d-8        [128, 16, 112, 112]              32
  InvertedResidual-9        [128, 16, 112, 112]               0
           Conv2d-10        [128, 64, 112, 112]           1,024
      BatchNorm2d-11        [128, 64, 112, 112]             128
             ReLU-12        [128, 64, 112, 112]               0
           Conv2d-13          [128, 64, 56, 56]             576
      BatchNorm2d-14          [128, 64,

In [11]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [12]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label)  
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 36.90 s
Accuracy : Train is 0.6231907894736842 , Valid is 0.720108695652174 ;  Loss : Train is  1.0373224609776548 ,Valid is 0.7657574933508168
覆盖最好的模型...
epoch: 1
The Model-Train-Time spent  0 min 30.28 s
Accuracy : Train is 0.8018092105263158 , Valid is 0.7527173913043478 ;  Loss : Train is  0.5666576388635134 ,Valid is 0.6973224308179773
覆盖最好的模型...
epoch: 2
The Model-Train-Time spent  0 min 29.36 s
Accuracy : Train is 0.8513980263157894 , Valid is 0.7398097826086957 ;  Loss : Train is  0.4244692261281766 ,Valid is 0.7491934014403302
epoch: 3
The Model-Train-Time spent  0 min 28.34 s
Accuracy : Train is 0.8889802631578947 , Valid is 0.8206521739130435 ;  Loss : Train is  0.3080285956985072 ,Valid is 0.5410911881405375
覆盖最好的模型...
epoch: 4
The Model-Train-Time spent  0 min 28.23 s
Accuracy : Train is 0.9365131578947368 , Valid is 0.8216711956521738 ;  Loss : Train is  0.18820368263282275 ,Valid is 0.6669212009595789
覆盖最好的模型...
epoch: 5
The Mod

The Model-Train-Time spent  0 min 28.12 s
Accuracy : Train is 0.9953947368421052 , Valid is 0.8372961956521738 ;  Loss : Train is  0.015479479102711928 ,Valid is 1.8381295800209045
epoch: 44
The Model-Train-Time spent  0 min 28.11 s
Accuracy : Train is 0.997203947368421 , Valid is 0.8335597826086957 ;  Loss : Train is  0.011402917114135467 ,Valid is 2.134634043859399
epoch: 45
The Model-Train-Time spent  0 min 28.12 s
Accuracy : Train is 0.9977796052631579 , Valid is 0.8410326086956521 ;  Loss : Train is  0.009205756434484532 ,Valid is 2.3088173192480337
epoch: 46
The Model-Train-Time spent  0 min 28.22 s
Accuracy : Train is 0.9989309210526316 , Valid is 0.8461277173913043 ;  Loss : Train is  0.0039015151070136774 ,Valid is 2.545113900433416
epoch: 47
The Model-Train-Time spent  0 min 28.10 s
Accuracy : Train is 0.9990131578947368 , Valid is 0.8525815217391304 ;  Loss : Train is  0.004542857712428821 ,Valid is 2.545507234075795
epoch: 48
The Model-Train-Time spent  0 min 28.20 s
Accura

Accuracy : Train is 1.0 , Valid is 0.8583559782608695 ;  Loss : Train is  6.440830858130204e-09 ,Valid is 2.8189657978389575
epoch: 89
The Model-Train-Time spent  0 min 28.70 s
Accuracy : Train is 1.0 , Valid is 0.8549592391304348 ;  Loss : Train is  3.638040078313727e-08 ,Valid is 2.8719796616098154
epoch: 90
The Model-Train-Time spent  0 min 28.75 s
Accuracy : Train is 1.0 , Valid is 0.8569972826086957 ;  Loss : Train is  1.575405660428499e-08 ,Valid is 2.8486036528711733
epoch: 91
The Model-Train-Time spent  0 min 28.91 s
Accuracy : Train is 1.0 , Valid is 0.8617527173913043 ;  Loss : Train is  1.463744985429864e-07 ,Valid is 2.7849948665370112
覆盖最好的模型...
epoch: 92
The Model-Train-Time spent  0 min 28.80 s
Accuracy : Train is 1.0 , Valid is 0.8590353260869565 ;  Loss : Train is  2.678581758549339e-07 ,Valid is 2.81573558372
epoch: 93
The Model-Train-Time spent  0 min 28.90 s
Accuracy : Train is 1.0 , Valid is 0.858016304347826 ;  Loss : Train is  7.292745929015311e-08 ,Valid is 2.80

The Model-Train-Time spent  0 min 29.41 s
Accuracy : Train is 0.9948190789473684 , Valid is 0.8114809782608695 ;  Loss : Train is  0.03575220813876704 ,Valid is 3.708824105884718
epoch: 134
The Model-Train-Time spent  0 min 29.46 s
Accuracy : Train is 0.9949013157894737 , Valid is 0.8362771739130435 ;  Loss : Train is  0.028566077841739906 ,Valid is 3.308982999428459
epoch: 135
The Model-Train-Time spent  0 min 28.38 s
Accuracy : Train is 0.9942434210526315 , Valid is 0.8254076086956521 ;  Loss : Train is  0.03931208064681605 ,Valid is 3.150428160377171
epoch: 136
The Model-Train-Time spent  0 min 28.34 s
Accuracy : Train is 0.9944901315789474 , Valid is 0.8325407608695652 ;  Loss : Train is  0.04266720107315402 ,Valid is 3.3532018868819526
epoch: 137
The Model-Train-Time spent  0 min 28.36 s
Accuracy : Train is 0.9914473684210526 , Valid is 0.8141983695652174 ;  Loss : Train is  0.049705141704333454 ,Valid is 3.1863518383191978
epoch: 138
The Model-Train-Time spent  0 min 28.35 s
Accu

Accuracy : Train is 0.9954769736842105 , Valid is 0.8318614130434783 ;  Loss : Train is  0.02881853627531152 ,Valid is 2.517314610273942
epoch: 177
The Model-Train-Time spent  0 min 28.24 s
Accuracy : Train is 0.9953947368421052 , Valid is 0.8305027173913043 ;  Loss : Train is  0.023465090029333766 ,Valid is 2.5457197116768877
epoch: 178
The Model-Train-Time spent  0 min 28.17 s
Accuracy : Train is 0.9947368421052631 , Valid is 0.8226902173913043 ;  Loss : Train is  0.03327836430582561 ,Valid is 2.4861758429071177
epoch: 179
The Model-Train-Time spent  0 min 28.20 s
Accuracy : Train is 0.9965460526315789 , Valid is 0.835258152173913 ;  Loss : Train is  0.017582698372241697 ,Valid is 2.7061091194982114
epoch: 180
The Model-Train-Time spent  0 min 28.25 s
Accuracy : Train is 0.9953947368421052 , Valid is 0.8199728260869565 ;  Loss : Train is  0.02681176610487072 ,Valid is 2.965999188630477
epoch: 181
The Model-Train-Time spent  0 min 28.38 s
Accuracy : Train is 0.9869243421052631 , Valid